# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
from secrets import IEX_CLOUD_API_TOKEN

symbol = 'AAPL'
base_url = 'https://sandbox.iexapis.com/stable'
api_url = f"{base_url}/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

{'companyName': 'Apple Inc', 'marketcap': 2486709498503, 'week52high': 153.3, 'week52low': 107.4, 'week52highSplitAdjustOnly': 158.32, 'week52lowSplitAdjustOnly': 105.8, 'week52change': 0.18291797550877376, 'sharesOutstanding': 17152954382, 'float': 0, 'avg10Volume': 73753725, 'avg30Volume': 71058744, 'day200MovingAvg': 137.54, 'day50MovingAvg': 152.03, 'employees': 151419, 'ttmEPS': 5.1, 'ttmDividendRate': 0.8830964545625183, 'dividendYield': 0.005833894711573437, 'nextDividendDate': '', 'exDividendDate': '2021-08-04', 'nextEarningsDate': '2021-10-17', 'peRatio': 28.55464515782432, 'beta': 1.540469622864751, 'maxChangePercent': 56.199263562523946, 'year5ChangePercent': 4.958542740108633, 'year2ChangePercent': 1.9832625364588137, 'year1ChangePercent': 0.20168617727728003, 'ytdChangePercent': 0.12335047189939194, 'month6ChangePercent': 0.23290018042493693, 'month3ChangePercent': 0.1782537980002198, 'month1ChangePercent': 0.0001497628352498692, 'day30ChangePercent': -0.002797957342317258

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data['year1ChangePercent']

0.20168617727728003

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

# for symbol_string in symbol_strings:
#     print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
def momentum_data():
    dataframe = pd.DataFrame(columns = my_columns)

    for symbols in symbol_strings:
        # print(symbols)
        api_url = f"{base_url}/stock/market/batch?symbols={symbols}&types=stats,price&token={IEX_CLOUD_API_TOKEN}"
        data = requests.get(api_url).json()
        # print(data['AAPL'])
        # print(data['AAPL']['stats'])

        for symbol in symbols.split(','):
            price = data[symbol]['price']
            # year1ChangePercent = data[symbol]['stats']['year1ChangePercent'],
            buy_count = np.nan

            dataframe = dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        data[symbol]['stats']['year1ChangePercent'],
                        buy_count
                    ],
                    index = my_columns
                ),
                ignore_index = True
            )

    return dataframe

final_dataframe = momentum_data()
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,175.93,0.796656,NaN
1,AAL,20.60,0.547664,NaN
2,AAP,210.91,0.374103,NaN
3,AAPL,149.13,0.2027,NaN
4,ABBV,121.20,0.360348,NaN
...,...,...,...,...
500,YUM,134.80,0.429834,NaN
501,ZBH,154.63,0.100436,NaN
502,ZBRA,586.48,1.137528,NaN
503,ZION,56.99,0.797628,NaN


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50] # Dump all but the first 50
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,83.82,2.306041,NaN
1,272,KSS,58.14,1.99929,NaN
2,441,TPR,42.37,1.962497,NaN
3,148,DVN,29.92,1.787997,NaN
4,344,NUE,123.24,1.713562,NaN
5,106,COF,175.13,1.683262,NaN
6,129,DFS,136.16,1.642939,NaN
7,23,ALB,239.30,1.602044,NaN
8,253,IVZ,25.16,1.504256,NaN
9,179,FCX,35.88,1.487216,NaN


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
def portfolio_input():
    global portfolio_size

    portfolio_size = 1000000 # input('Enter the value of your portfolio:')

    try:
        float(portfolio_size)
    except ValueError:
        print('Portfolio size should be an integer.')
        portfolio_size = input('Enter the value of your portfolio:')
        float(portfolio_size)

portfolio_input()
portfolio_size

1000000

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    buy_count =  math.floor(position_size / final_dataframe.loc[i, 'Price'])
    final_dataframe.loc[i, 'Number of Shares to Buy'] = buy_count

final_dataframe

/opt/homebrew/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,83.82,2.306041,238.0
1,272,KSS,58.14,1.99929,343.0
2,441,TPR,42.37,1.962497,472.0
3,148,DVN,29.92,1.787997,668.0
4,344,NUE,123.24,1.713562,162.0
5,106,COF,175.13,1.683262,114.0
6,129,DFS,136.16,1.642939,146.0
7,23,ALB,239.30,1.602044,83.0
8,253,IVZ,25.16,1.504256,794.0
9,179,FCX,35.88,1.487216,557.0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [11]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

def hq_momentum_data():
    dataframe = pd.DataFrame(columns = hqm_columns)

    for symbols in symbol_strings:
        # print(symbols)
        api_url = f"{base_url}/stock/market/batch?symbols={symbols}&types=stats,price&token={IEX_CLOUD_API_TOKEN}"
        data = requests.get(api_url).json()
        # print(data['AAPL'])
        # print(data['AAPL']['stats'])

        for symbol in symbols.split(','):
            price = data[symbol]['price']
            # year1ChangePercent = data[symbol]['stats']['year1ChangePercent'],
            buy_count = np.nan

            dataframe = dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        buy_count,
                        data[symbol]['stats']['year1ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month6ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month3ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month1ChangePercent'], 
                        np.nan,
                        np.nan,
                    ],
                    index = hqm_columns,
                    
                ),
                ignore_index = True
            )

    # https://stackoverflow.com/questions/65174575/typeerror-not-supported-between-instances-of-nonetype-and-float
    return dataframe.fillna(0)

hqm_dataframe = hq_momentum_data()
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,178.03,0.0,0.774237,0.0,0.433952,0.0,0.307089,0.0,0.143844,0.0,0.0
1,AAL,20.20,0.0,0.545314,0.0,-0.035337,0.0,-0.135060,0.0,-0.050663,0.0,0.0
2,AAP,210.98,0.0,0.391289,0.0,0.337497,0.0,0.141363,0.0,-0.012794,0.0,0.0
3,AAPL,152.68,0.0,0.198099,0.0,0.230478,0.0,0.171205,0.0,0.000153,0.0,0.0
4,ABBV,123.10,0.0,0.347973,0.0,0.150424,0.0,0.064517,0.0,0.019477,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.20,0.0,0.416267,0.0,0.292799,0.0,0.121184,0.0,0.095060,0.0,0.0
501,ZBH,152.88,0.0,0.102775,0.0,-0.084380,0.0,-0.102749,0.0,-0.061598,0.0,0.0
502,ZBRA,601.20,0.0,1.146528,0.0,0.177110,0.0,0.171577,0.0,0.067561,0.0,0.0
503,ZION,56.98,0.0,0.778580,0.0,0.054842,0.0,0.019918,0.0,0.124480,0.0,0.0


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [12]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        column_return = f'{time_period} Price Return'
        column_percentile = f'{time_period} Return Percentile'

        # print(hqm_dataframe.convert_dtypes().dtypes)
        # print(hqm_dataframe[column_return])
        price_return_all = pd.to_numeric(hqm_dataframe[column_return].astype(float))
        price_return_single = hqm_dataframe.loc[row, column_return]

        percentile = score(price_return_all, price_return_single) / 100
        hqm_dataframe.loc[row, column_percentile] = percentile

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,178.03,0.0,0.774237,0.853465,0.433952,0.946535,0.307089,0.970297,0.143844,0.962376,0.0
1,AAL,20.20,0.0,0.545314,0.691089,-0.035337,0.110891,-0.135060,0.065347,-0.050663,0.118812,0.0
2,AAP,210.98,0.0,0.391289,0.510891,0.337497,0.849505,0.141363,0.819802,-0.012794,0.267327,0.0
3,AAPL,152.68,0.0,0.198099,0.259406,0.230478,0.685149,0.171205,0.861386,0.000153,0.330693,0.0
4,ABBV,123.10,0.0,0.347973,0.447525,0.150424,0.499010,0.064517,0.617822,0.019477,0.463366,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.20,0.0,0.416267,0.550495,0.292799,0.776238,0.121184,0.764356,0.095060,0.843564,0.0
501,ZBH,152.88,0.0,0.102775,0.168317,-0.084380,0.067327,-0.102749,0.093069,-0.061598,0.093069,0.0
502,ZBRA,601.20,0.0,1.146528,0.956436,0.177110,0.574257,0.171577,0.863366,0.067561,0.718812,0.0
503,ZION,56.98,0.0,0.778580,0.857426,0.054842,0.275248,0.019918,0.465347,0.124480,0.934653,0.0


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [13]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
        hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,178.03,0.0,0.774237,0.853465,0.433952,0.946535,0.307089,0.970297,0.143844,0.962376,0.933168
1,AAL,20.20,0.0,0.545314,0.691089,-0.035337,0.110891,-0.135060,0.065347,-0.050663,0.118812,0.246535
2,AAP,210.98,0.0,0.391289,0.510891,0.337497,0.849505,0.141363,0.819802,-0.012794,0.267327,0.611881
3,AAPL,152.68,0.0,0.198099,0.259406,0.230478,0.685149,0.171205,0.861386,0.000153,0.330693,0.534158
4,ABBV,123.10,0.0,0.347973,0.447525,0.150424,0.499010,0.064517,0.617822,0.019477,0.463366,0.506931
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.20,0.0,0.416267,0.550495,0.292799,0.776238,0.121184,0.764356,0.095060,0.843564,0.733663
501,ZBH,152.88,0.0,0.102775,0.168317,-0.084380,0.067327,-0.102749,0.093069,-0.061598,0.093069,0.105446
502,ZBRA,601.20,0.0,1.146528,0.956436,0.177110,0.574257,0.171577,0.863366,0.067561,0.718812,0.778218
503,ZION,56.98,0.0,0.778580,0.857426,0.054842,0.275248,0.019918,0.465347,0.124480,0.934653,0.633168


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [14]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,238.70,0.0,1.665895,0.988119,0.550285,0.982178,0.490419,1.000000,0.230616,0.996040,0.991584
1,IT,309.00,0.0,1.381501,0.976238,0.746017,0.994059,0.330703,0.972277,0.183973,0.986139,0.982178
2,NUE,123.89,0.0,1.721202,0.992079,1.002790,1.000000,0.214263,0.908911,0.280820,1.000000,0.975248
3,FTNT,315.21,0.0,1.334947,0.974257,0.860433,0.998020,0.440045,0.992079,0.114257,0.908911,0.968317
4,NVDA,231.03,0.0,0.775284,0.855446,0.678146,0.992079,0.424537,0.990099,0.140256,0.956436,0.948515
5,CARR,57.99,0.0,0.919946,0.902970,0.617354,0.988119,0.295854,0.964356,0.115718,0.910891,0.941584
6,A,178.03,0.0,0.774237,0.853465,0.433952,0.946535,0.307089,0.970297,0.143844,0.962376,0.933168
7,LB,82.17,0.0,2.353514,1.000000,0.847902,0.996040,0.218387,0.916832,0.081449,0.784158,0.924257
8,DVN,29.31,0.0,1.775276,0.994059,0.395031,0.914851,0.140397,0.813861,0.145457,0.964356,0.921782
9,WST,466.70,0.0,0.623378,0.760396,0.604823,0.986139,0.289469,0.960396,0.168561,0.976238,0.920792


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [15]:
portfolio_input()
portfolio_size

1000000

In [16]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for row in hqm_dataframe.index:
    price = hqm_dataframe.loc[row, 'Price']
    buy_count = math.floor(position_size / price)
    hqm_dataframe.loc[row, 'Number of Shares to Buy'] = buy_count

hqm_dataframe

/opt/homebrew/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,238.70,83.0,1.665895,0.988119,0.550285,0.982178,0.490419,1.000000,0.230616,0.996040,0.991584
1,IT,309.00,64.0,1.381501,0.976238,0.746017,0.994059,0.330703,0.972277,0.183973,0.986139,0.982178
2,NUE,123.89,161.0,1.721202,0.992079,1.002790,1.000000,0.214263,0.908911,0.280820,1.000000,0.975248
3,FTNT,315.21,63.0,1.334947,0.974257,0.860433,0.998020,0.440045,0.992079,0.114257,0.908911,0.968317
4,NVDA,231.03,86.0,0.775284,0.855446,0.678146,0.992079,0.424537,0.990099,0.140256,0.956436,0.948515
5,CARR,57.99,344.0,0.919946,0.902970,0.617354,0.988119,0.295854,0.964356,0.115718,0.910891,0.941584
6,A,178.03,112.0,0.774237,0.853465,0.433952,0.946535,0.307089,0.970297,0.143844,0.962376,0.933168
7,LB,82.17,243.0,2.353514,1.000000,0.847902,0.996040,0.218387,0.916832,0.081449,0.784158,0.924257
8,DVN,29.31,682.0,1.775276,0.994059,0.395031,0.914851,0.140397,0.813861,0.145457,0.964356,0.921782
9,WST,466.70,42.0,0.623378,0.760396,0.604823,0.986139,0.289469,0.960396,0.168561,0.976238,0.920792


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [17]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [18]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format': '$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format': '0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format': '0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [19]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template],
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [20]:
writer.save()